In [4]:
from scipy.integrate import odeint
import numpy as np
import networkx as nx
import pandas as pd
from numba import jit
import pickle
from collections import defaultdict

def Deqs(x, t, A, rho, nu, M0):

    N = A.shape[0]
    numerator = (np.multiply(M0, 1 + np.sum(A, axis=1)) + np.multiply(nu, x) + np.multiply((nu+1),(A.dot(x))))
    denominator = ( np.multiply(1.*rho,t) + (A + np.identity(N)).dot(M0 + np.multiply((nu+1), x)))
    #dxdt = numerator/denominator
    dxdt = numerator/denominator
    
    return dxdt

def get_heaps_UMT(G, rho, nu, M0, tmax=10000):
    
    A = nx.to_numpy_array(G)
    N = A.shape[0]

    x0 = [0]*N
    t = np.arange(tmax)

    params = (A, rho, nu, M0)
    res = odeint(Deqs, x0, t, args=params)

    analytical_heaps={i: res[:,i] for i in range(N)} 
    
    return t, analytical_heaps

@jit
def get_logD_fit(analytical_heaps, tmax):
    betas={}
    for node, yana in analytical_heaps.iteritems():
        Dmax = yana[tmax-1]
        beta = 1.*np.log10(Dmax)/np.log10(tmax)
        betas[node]=beta
    return betas

@jit
def get_moving_fit(analytical_heaps, tfit):
    betas_dict = defaultdict(list)
    for ti in range(len(tfit))[:-1]:
        tmax = int(tfit[ti+1])
        betas = get_logD_fit(analytical_heaps, tmax)
        for node, beta_node in betas.iteritems():
            betas_dict[node].append(beta_node)
    return betas_dict

# Toy synthetic networks (4 nodes) -- Fig. 3
In this notebook I look at the Heaps behavior at transient times [via Eq.(4)] for different toy networks

Net1: star

In [3]:
rho = 6
nu = 3
M0 = nu+1
tmax = 100000000

G = nx.DiGraph()
G.add_edge(0,1)
G.add_edge(0,2)
G.add_edge(0,3)

t, analytical_heaps = get_heaps_UMT(G, rho, nu, M0, tmax)

In [7]:
tfit = np.logspace(1, 8, 25)
betas_dict = get_moving_fit(analytical_heaps, tfit)

In [8]:
#Saving
df = pd.DataFrame()
df['time'] = tfit[1:]
df['beta_n1'] = betas_dict[0]
df['beta_n2'] = betas_dict[1]
df['beta_n3'] = betas_dict[2]
df['beta_n4'] = betas_dict[3]

filename = "../results/Ready_to_plot/Heaps_panel_transient_net1.csv"
df.to_csv(filename, header=True, index=None, sep=',')

Net 2: chain

In [9]:
rho = 6
nu = 3
M0 = nu+1
tmax = 100000000

G = nx.DiGraph()
G.add_edge(0,1)
G.add_edge(1,2)
G.add_edge(2,3)

t, analytical_heaps = get_heaps_UMT(G, rho, nu, M0, tmax)

In [10]:
tfit = np.logspace(1, 8, 25)
betas_dict = get_moving_fit(analytical_heaps, tfit)

In [11]:
#Saving
df = pd.DataFrame()
df['time'] = tfit[1:]
df['beta_n1'] = betas_dict[0]
df['beta_n2'] = betas_dict[1]
df['beta_n3'] = betas_dict[2]
df['beta_n4'] = betas_dict[3]

filename = "../results/Ready_to_plot/Heaps_panel_transient_net2.csv"
df.to_csv(filename, header=True, index=None, sep=',')

Net 3: 4 nodes with triangle

In [12]:
rho = 6
nu = 3
M0 = nu+1
tmax = 100000000

G = nx.DiGraph()
G.add_edge(0,1)
G.add_edge(1,2)
G.add_edge(2,3)
G.add_edge(3,1)

t, analytical_heaps = get_heaps_UMT(G, rho, nu, M0, tmax)

In [13]:
tfit = np.logspace(1, 8, 25)
betas_dict = get_moving_fit(analytical_heaps, tfit)

In [14]:
#Saving
df = pd.DataFrame()
df['time'] = tfit[1:]
df['beta_n1'] = betas_dict[0]
df['beta_n2'] = betas_dict[1]
df['beta_n3'] = betas_dict[2]
df['beta_n4'] = betas_dict[3]

filename = "../results/Ready_to_plot/Heaps_panel_transient_net3.csv"
df.to_csv(filename, header=True, index=None, sep=',')

Net 4: 4 nodes not sym 

In [15]:
rho = 6
nu = 3
M0 = nu+1
tmax = 100000000

G = nx.DiGraph()
G.add_edge(0,1)
G.add_edge(1,2)
G.add_edge(2,3)
G.add_edge(1,3)

t, analytical_heaps = get_heaps_UMT(G, rho, nu, M0, tmax)

In [16]:
tfit = np.logspace(1, 8, 25)
betas_dict = get_moving_fit(analytical_heaps, tfit)

In [17]:
#Saving
df = pd.DataFrame()
df['time'] = tfit[1:]
df['beta_n1'] = betas_dict[0]
df['beta_n2'] = betas_dict[1]
df['beta_n3'] = betas_dict[2]
df['beta_n4'] = betas_dict[3]

filename = "../results/Ready_to_plot/Heaps_panel_transient_net4.csv"
df.to_csv(filename, header=True, index=None, sep=',')

Net 5: 4 nodes with triangle (out)

In [18]:
rho = 6
nu = 3
M0 = nu+1
tmax = 100000000

G = nx.DiGraph()
G.add_edge(0,1)
G.add_edge(1,2)
G.add_edge(1,3)
G.add_edge(3,0)

t, analytical_heaps = get_heaps_UMT(G, rho, nu, M0, tmax)

In [19]:
tfit = np.logspace(1, 8, 25)
betas_dict = get_moving_fit(analytical_heaps, tfit)

In [20]:
#Saving
df = pd.DataFrame()
df['time'] = tfit[1:]
df['beta_n1'] = betas_dict[0]
df['beta_n2'] = betas_dict[1]
df['beta_n3'] = betas_dict[2]
df['beta_n4'] = betas_dict[3]

filename = "../results/Ready_to_plot/Heaps_panel_transient_net5.csv"
df.to_csv(filename, header=True, index=None, sep=',')